NOTE: First part of notebook is just SAM experimentation, code mostly taken from SAM tutorial published by Roboflow:
https://blog.roboflow.com/how-to-use-segment-anything-model-sam/

In [1]:
!nvidia-smi

Thu Mar  7 03:11:44 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 551.61                 Driver Version: 551.61         CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3070 ...  WDDM  |   00000000:01:00.0 Off |                  N/A |
| N/A   50C    P8             12W /  100W |      10MiB /   8192MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
import os
HOME = os.getcwd()
print("HOME:", HOME)

In [ ]:
!mkdir -p {HOME}/weights
!wget -q https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth -P {HOME}/weights

In [ ]:
import os

CHECKPOINT_PATH = os.path.join(HOME, "weights", "sam_vit_h_4b8939.pth")
print(CHECKPOINT_PATH, "; exist:", os.path.isfile(CHECKPOINT_PATH))

Load Dataset

Experimenting with one image for now: https://simplify101.com/wp-content/uploads/2012/08/inside-refrigerator.jpg


In [ ]:
img_path = "C:/Users/gener/Desktop/Refrigerator/images"

img1 = img_path + "/fridge1.jpg"

import numpy as np
import torch
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor
import cv2
import supervision as sv

DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
MODEL_TYPE = "vit_h"

sam = sam_model_registry[MODEL_TYPE](checkpoint=CHECKPOINT_PATH).to(device=DEVICE)


IMAGE_PATH = img1
print(IMAGE_PATH)

In [ ]:
dims = cv2.imread(IMAGE_PATH)

dims = cv2.cvtColor(dims, cv2.COLOR_BGR2RGB)

print(dims.shape)

In [ ]:
mask_generator = SamAutomaticMaskGenerator(sam)


image_bgr = cv2.imread(IMAGE_PATH)
image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)

sam_result = mask_generator.generate(image_rgb)

mask_annotator = sv.MaskAnnotator(color_lookup=sv.ColorLookup.INDEX)

detections = sv.Detections.from_sam(sam_result=sam_result)

annotated_image = mask_annotator.annotate(scene=image_bgr.copy(), detections=detections)

sv.plot_images_grid(
    images=[image_bgr, annotated_image],
    grid_size=(1, 2),
    titles=['source image', 'segmented image']
)

In [ ]:
mask_predictor = SamPredictor(sam)

image_bgr = cv2.imread(IMAGE_PATH)
image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)
mask_predictor.set_image(image_rgb)

box = np.array([150, 160, 250, 240])
masks, scores, logits = mask_predictor.predict(
    box=box,
    multimask_output=True
)

YOLOv8
=

Contains code for training YOLOv8 model as well as converting output for use in search. Training code was adapted from the Ultralytics documentation:
https://docs.ultralytics.com/modes/train/

In [ ]:
from datasets import load_dataset

dataset = load_dataset("arjunpatel1234/Recipe_Ingredients")
dataset.save_to_disk("images/dataset.hf")

In [1]:
import torch
import torchvision
torch.cuda.is_available()

True

In [2]:
torchvision.__version__ 

'0.17.1+cu121'

In [3]:
torch.__version__ 

'2.2.1+cu121'

In [4]:
from roboflow import Roboflow
rf = Roboflow(api_key="M2leDGQkwkb7L9j0KmKx")
project = rf.workspace("cs131").project("fridge-5ep2w")
version = project.version(1)
dataset = version.download("yolov5")


loading Roboflow workspace...
loading Roboflow project...


In [ ]:
from ultralytics import YOLO
model = YOLO("yolov8s.pt") # load the model
model.to('cuda')
results = model.train(data="C:/Users/gener/Desktop/Refrigerator/fridge-1/data.yaml", imgsz=640, epochs=100, batch=12)

engine\trainer: task=detect, mode=train, model=yolov8s.pt, data=C:/Users/gener/Desktop/Refrigerator/fridge-1/data.yaml, epochs=100, time=None, patience=100, batch=12, imgsz=640, save=True, save_period=-1, cache=False, device=cuda:0, workers=8, project=None, name=train18, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line_width=None, format=torchscript, keras

train: Scanning C:\Users\gener\Desktop\Refrigerator\fridge-1\train\labels.cache... 5882 images, 117 backgrounds, 0 corr
val: Scanning C:\Users\gener\Desktop\Refrigerator\fridge-1\valid\labels.cache... 389 images, 2 backgrounds, 0 corrupt: 


Plotting labels to C:\Users\gener\PycharmProjects\pangolin\runs\detect\train18\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.00027, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.00046875), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to C:\Users\gener\PycharmProjects\pangolin\runs\detect\train18
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      1/100      3.37G     0.7839      2.913      1.274          4        640: 100%|██████████| 491/491 [01:18<00:00,  
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 17/17 [00:03

                   all        389       1261      0.127       0.11     0.0817     0.0602


...

                   all        389       1261        0.2   0.000862        0.1     0.0201



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     90/100      3.24G     0.3341     0.3077     0.9371         15        640: 100%|██████████| 491/491 [01:28<00:00,  
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 17/17 [00:03

                   all        389       1261        0.2   0.000862        0.1     0.0201


Closing dataloader mosaic


In [ ]:
results = model("./image.png")

In [2]:
from ultralytics import YOLO
trained = YOLO("C:/Users/gener/Desktop/Refrigerator/train6/weights/best.pt")

In [16]:
from PIL import Image

results = trained("veggies2.jpg", conf=0.1)

for i, r in enumerate(results):
    # Plot results image
    im_bgr = r.plot()  # BGR-order numpy array
    im_rgb = Image.fromarray(im_bgr[..., ::-1])  # RGB-order PIL image

    # Show results to screen (in supported environments)
    r.show()



image 1/1 C:\Users\gener\Desktop\Refrigerator\veggies2.jpg: 384x640 1 Bell pepper, 1 Radish, 1 Tomato, 40.0ms
Speed: 0.0ms preprocess, 40.0ms inference, 8.0ms postprocess per image at shape (1, 3, 384, 640)


In [47]:
import torch
import torchvision
# torch.cuda.is_available()

torchvision.__version__ 
torch.__version__ 

from ultralytics import YOLO
trained = YOLO("C:/Users/gener/Desktop/Refrigerator/train6/weights/best.pt")

from PIL import Image

results = trained("veggies2.jpg", conf=0.01, save_txt=True)

for i, r in enumerate(results):
    # Plot results image
    im_bgr = r.plot()  # BGR-order numpy array
    im_rgb = Image.fromarray(im_bgr[..., ::-1])  # RGB-order PIL image

    # Show results to screen (in supported environments)
    r.show()



image 1/1 C:\Users\gener\Desktop\Refrigerator\veggies2.jpg: 384x640 1 Bell pepper, 1 Cabbage, 1 Chili pepper, 1 Corn, 1 Garlic, 1 Radish, 1 Tomato, 46.9ms
Speed: 0.0ms preprocess, 46.9ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)
Results saved to C:\Users\gener\PycharmProjects\pangolin\runs\detect\predict4
1 label saved to C:\Users\gener\PycharmProjects\pangolin\runs\detect\predict4\labels


In [14]:
print(results[0])

ultralytics.engine.results.Results object with attributes:

boxes: ultralytics.engine.results.Boxes object
keypoints: None
masks: None
names: {0: 'Apple', 1: 'Banana', 2: 'Beetroot', 3: 'Bell pepper', 4: 'Cabbage', 5: 'Carrot', 6: 'Cauliflower', 7: 'Chili pepper', 8: 'Corn', 9: 'Cucumber', 10: 'Eggplant', 11: 'Garlic', 12: 'Ginger', 13: 'Grapes', 14: 'Jalapeno', 15: 'Kiwi', 16: 'Lemon', 17: 'Lettuce', 18: 'Mango', 19: 'Onion', 20: 'Orange', 21: 'Pear', 22: 'Peas', 23: 'Pineapple', 24: 'Pomegranate', 25: 'Potato', 26: 'Radish', 27: 'Soybean', 28: 'Spinach', 29: 'Sweet potato', 30: 'Tomato', 31: 'Turnip', 32: 'Watermelon'}
obb: None
orig_img: array([[[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 25

In [22]:
print(results[0].names)

{0: 'Apple', 1: 'Banana', 2: 'Beetroot', 3: 'Bell pepper', 4: 'Cabbage', 5: 'Carrot', 6: 'Cauliflower', 7: 'Chili pepper', 8: 'Corn', 9: 'Cucumber', 10: 'Eggplant', 11: 'Garlic', 12: 'Ginger', 13: 'Grapes', 14: 'Jalapeno', 15: 'Kiwi', 16: 'Lemon', 17: 'Lettuce', 18: 'Mango', 19: 'Onion', 20: 'Orange', 21: 'Pear', 22: 'Peas', 23: 'Pineapple', 24: 'Pomegranate', 25: 'Potato', 26: 'Radish', 27: 'Soybean', 28: 'Spinach', 29: 'Sweet potato', 30: 'Tomato', 31: 'Turnip', 32: 'Watermelon'}


In [23]:
# Extract text file info
import json

class_list = []

with open('C:/Users/gener/PycharmProjects/pangolin/runs/detect/predict2/labels/veggies2.txt') as f:
    lines = f.readlines() # list containing lines of file
    columns = [] # To store column names

    i = 1
    for line in lines:
        line = line.strip() 
        splitted_line = line.split()
        
        class_list.append(splitted_line[0]) 
        
print(class_list)
results_dict = results[0].names

ingredients_dict = {}
for i in class_list:
    index = int(i)
    ingred_name = results_dict[index]
    if ingred_name not in ingredients_dict:
        ingredients_dict[ingred_name] = 0
    ingredients_dict[ingred_name] += 1
    
print(ingredients_dict)


['30', '3', '26', '8', '4', '11', '7']
{'Tomato': 1, 'Bell pepper': 1, 'Radish': 1, 'Corn': 1, 'Cabbage': 1, 'Garlic': 1, 'Chili pepper': 1}


In [43]:
print(ingredients_dict.keys())
ingredients_str = ''
first = True
j = 0
for ing in ingredients_dict.keys():
    if j > 2:
        break
    if not first:
        ingredients_str += ','
    ingredients_str += ing.lower()
    first = False
    j += 1
print(ingredients_str)
food = ingredients_str

dict_keys(['Tomato', 'Bell pepper', 'Radish', 'Corn', 'Cabbage', 'Garlic', 'Chili pepper'])
tomato,bell pepper,radish


In [39]:
import pandas as pd
import pickle 
from itertools import compress

d = None
df = None
freq = {}

def process_1K():
    df = pd.read_csv('1K_dataset.csv')
    df["IGS"] = df["NER"].apply(lambda x: set(x.lower().replace('[', '').replace(']', '').replace('\"', '').replace(', ', ',').split(",")))
    d = df["IGS"].values
    np.save('1K_IGS', d)
    return d

def process_full():
    df = pd.read_csv('full_dataset.csv')
    df["IGS"] = df["NER"].apply(lambda x: set(x.lower().replace('[', '').replace(']', '').replace('\"', '').replace(', ', ',').split(",")))
    d = df["IGS"].values
    np.save('full_IGS', d)
    return d

def process_optimized_1K():
    df = pd.read_csv('1K_dataset.csv')
    freq = {}
    for i in range(df.shape[0]):
        for ing in df["NER"][i].lower().replace('[', '').replace(']', '').replace('\"', '').replace(', ', ',').split(","):
            if ing in freq: freq[ing].append(i)
            else: freq[ing] = [i]
        print(i)
    with open('prcomputed_indices_1K.pkl', 'wb') as f:
        pickle.dump(freq, f)
    return df, freq

def process_optimized_full():
    df = pd.read_csv('full_dataset.csv')
    freq = {}
    for i in range(df.shape[0]):
        for ing in df["NER"][i].lower().replace('[', '').replace(']', '').replace('\"', '').replace(', ', ',').split(","):
            if ing in freq: freq[ing].append(i)
            else: freq[ing] = [i]
#         print(i)
    with open('prcomputed_indices_full.pkl', 'wb') as f:
        pickle.dump(freq, f)
    return df, freq

def initialize_1K():
    df = pd.read_csv('1K_dataset.csv')
    d = np.load('1k_IGS.npy', allow_pickle=True)
    return df, d

def initialize_full():
    df = pd.read_csv('full_dataset.csv')
    d = np.load('full_IGS.npy', allow_pickle=True)
    return df, d

def initialize_optimized_1K():
    df = pd.read_csv('full_dataset.csv')
    with open('prcomputed_indices_1K.pkl', 'rb') as f:
        freq = pickle.load(f)
    return df, freq

def initialize_optimized_full():
    df = pd.read_csv('full_dataset.csv')
    with open('prcomputed_indices_full.pkl', 'rb') as f:
        freq = pickle.load(f)
    return df, freq


def get_indices(ing_list):
    ingredients = set(ing_list)
    return np.where([ingredients.issubset(x) for x in d])

def get_indices_optimized(ing_list):
    if set(ing_list).issubset(freq.keys()):
        base = freq[ing_list[0]]
        for ing in ing_list:
            base = np.intersect1d(base, freq[ing])
        return base
    else: return []

def get_recipes(ing_list):
    indices = get_indices(ing_list)
    return df.loc[indices[0], ["title", "link"]]

def get_recipes_optimized(ing_list):
    indices = get_indices_optimized(ing_list)
    return df.loc[indices, ["title", "link"]]


# df, d = initialize_1K()
# while True:
#     food = input("Enter a food (Use commas w/o space): ")
#     print(get_recipes(food.split(",")))

In [44]:
process_optimized_full()
print("process optimized done")
df, freq = initialize_optimized_full()
print("initialization done")
print(get_recipes_optimized(food.split(",")))

process optimized done
initialization done
                title                                             link
663682  Veggie Snacks  www.cookbooks.com/Recipe-Details.aspx?id=687300


In [46]:
food = "watermelon,sugar"
print(get_recipes_optimized(food.split(",")))

                                 title  \
25047               Watermelon Pickles   
37043          Watermelon Rind Pickles   
67514                 Melon Ball Salad   
74229          Watermelon Rind Pickles   
122191              Watermelon Pickles   
...                                ...   
2221780     Watermelon Mochaccino Cake   
2224510        Vietnamese Tuna Seviche   
2226817                   Fruity Salsa   
2228513  Watermelon White Pepper Shrub   
2230044           Watermelon Margarita   

                                                      link  
25047       www.cookbooks.com/Recipe-Details.aspx?id=79242  
37043       www.cookbooks.com/Recipe-Details.aspx?id=64665  
67514       www.cookbooks.com/Recipe-Details.aspx?id=49990  
74229     www.cookbooks.com/Recipe-Details.aspx?id=1012571  
122191     www.cookbooks.com/Recipe-Details.aspx?id=855624  
...                                                    ...  
2221780  cookpad.com/us/recipes/348211-watermelon-mocha...  
2224510